In [ ]:
import pandas as pd
import sys

In [ ]:
df_raw = pd.read_csv('cdr.csv', sep=';', header=None)

In [ ]:
df_raw.tail()

In [ ]:
df_raw1 = df_raw[[0,1,2,3,5,6,7,10,11]].copy()
df_raw1.columns = ['Destino', 'Data', 'Conta Voip', 'Numero', 'Duracao', 'Usuario', 'Tronco','Compra', 'Venda']
df_raw1.head()

In [ ]:
def split_tronco(str):
    lst = str.split('_')
    if len(lst) <3:
        return 'Entrada'
    
    if lst[2] == 'Cel':
        return 'Movel'
    
    elif lst[2] == 'Fixo':
        return 'Fixo'
    
    else:
        return 'outro'
df_raw1['Tipo_Ligação'] = df_raw1.Tronco.apply(split_tronco)
df_raw1.tail()

In [ ]:
def busca_cliente(str):
    cli = df_raw1[df_raw1['Usuario'] == str]
    return cli

In [ ]:
brshield = busca_cliente('brshield')

In [ ]:
def cal_lucro(str):
    c = str.Compra.sum()
    v = str.Venda.sum()
    l = str.Venda.sum() - str.Compra.sum()
    print('''Somatoria dos valores :
    -> Compra R$ {:.2f} 
    -> Venda R$ {:.2f}
    -> Lucro R$ {:.2f}
    '''.format(c,v,l))

In [ ]:
cal_lucro(brshield)

In [ ]:
cal_lucro(brshield[selecao])

In [ ]:
def cal_clientes(df,i,f):
    '''
    Funcçao usada para calcular valores de custo, venda e lucro de uma tabela
    df -> Dataframe a ser analisado
    i -> Data inicio da busca (ex: '2019-01-01' ou '2019' ou '2019-01-01 00:00:00')
    f - Data final da busca (ex: '2019-01-01' ou '2019' ou '2019-01-01 00:00:00')
    '''
    cli = df['Usuario'].unique()
    p = (df['Data'] >= i) & (df['Data'] <= f)
    for c in cli :
        _df = df[df['Usuario'] == c]
        print(f'O cliente {c} no periodo de {i} ate {f} : ')
        cal_lucro(_df[p])

In [ ]:
cal_clientes(df_raw1[df_raw1["Usuario"] == 'Escritorio'],'2019-01-01','2020-01-01')

In [ ]:
cal_clientes(df_raw1,'2019','2020')